# 1.  Extracting all headers from "http://en.wikipedia.org/wiki/Main_Page?"

In [47]:
# importing reqd libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [2]:
# Displaying all the header tags
def AllHeaders(url):
    if requests.get(url):
        html =urlopen(url)
        bs = BeautifulSoup(html, "html.parser")
        titles = bs.find_all(['h1', 'h2','h3','h4','h5','h6'])
        print('Listing all header tags :', *titles, sep='\n')
        
    else:
        print("Web Scraping Denied")
    
    

In [3]:
AllHeaders("http://en.wikipedia.org/wiki/Main_Page?")

Listing all header tags :
<h1 class="firstHeading" id="firstHeading">Main Page</h1>
<h2 class="mp-h2" id="mp-tfa-h2"><span id="From_today.27s_featured_article"></span><span class="mw-headline" id="From_today's_featured_article">From today's featured article</span></h2>
<h2 class="mp-h2" id="mp-dyk-h2"><span class="mw-headline" id="Did_you_know_...">Did you know ...</span></h2>
<h2 class="mp-h2" id="mp-itn-h2"><span class="mw-headline" id="In_the_news">In the news</span></h2>
<h2 class="mp-h2" id="mp-otd-h2"><span class="mw-headline" id="On_this_day">On this day</span></h2>
<h2 class="mp-h2" id="mp-tfp-h2"><span id="Today.27s_featured_picture"></span><span class="mw-headline" id="Today's_featured_picture">Today's featured picture</span></h2>
<h2 class="mp-h2" id="mp-other"><span class="mw-headline" id="Other_areas_of_Wikipedia">Other areas of Wikipedia</span></h2>
<h2 class="mp-h2" id="mp-sister"><span id="Wikipedia.27s_sister_projects"></span><span class="mw-headline" id="Wikipedia's_s

# 2. IMDB Top rated  100 Movies

In [16]:
#Lodaing the libraries
import numpy as np
import pandas as pd
from time import sleep
from random import randint
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [5]:
def Top100Movies(url):
    # Creating the lists we want to write into
    titles = []
    years = []
    imdb_ratings = []
    #pages we need is the first page with 1-50 and second from 51-100
    pages = np.arange(1, 101, 50)
    
    # Getting English translated titles from the movies
    headers = {'Accept-Language': 'en-US, en;q=0.5'}

    if requests.get(url):
        
        #initialising the page with main page having no page count in url
        page = requests.get(url , headers=headers)
        soup = BeautifulSoup(page.text, 'html.parser')
    
        # Aiming the part of the html we want to get the information from
        movie_div = soup.find_all('div', class_='lister-item mode-advanced')
        for container in movie_div:
            # Scraping the movie's name
            name = container.h3.a.text
            titles.append(name)
        
            # Scraping the movie's year
            year = container.h3.find('span', class_='lister-item-year').text
            years.append(year)
        
        
            # Scraping the rating
            imdb = float(container.strong.text)
            imdb_ratings.append(imdb)
            
        for page in pages:
            # Getting the contents from the each url
            page=requests.get(url + '&start='  + str(page) + '&ref_=adv_nxt', headers=headers)
            soup = BeautifulSoup(page.text, 'html.parser')
    
            # Aiming the part of the html we want to get the information from
            movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    
            # Controling the loop’s rate by pausing the execution of the loop for a specified amount of time
            # Waiting time between requests for a number between 2-10 seconds
        sleep(randint(2,10))
        
        for container in movie_div:
            # Scraping the movie's name
            name = container.h3.a.text
            titles.append(name)
        
            # Scraping the movie's year
            year = container.h3.find('span', class_='lister-item-year').text
            years.append(year)
        
        
            # Scraping the rating
            imdb = float(container.strong.text)
            imdb_ratings.append(imdb)
            
        #saving the data in Dataframe
        movies = pd.DataFrame({'MovieName':titles,
                       'Year of Release':years,
                       'IMDB Rating':imdb_ratings,
                       })

            # Cleaning 'year' column
        movies['Year of Release'] = movies['Year of Release'].str.extract('(\d+)').astype(int)
        print(movies)
        movies.to_csv("Top100Movies.csv")
    else:
        print("Web Scraping denied")


    
   

In [6]:
Top100Movies("https://www.imdb.com/search/title/?groups=top_100&sort=user_rating,desc")

                   MovieName  Year of Release  IMDB Rating
0   The Shawshank Redemption             1994          9.3
1              The Godfather             1972          9.2
2            The Dark Knight             2008          9.0
3     The Godfather: Part II             1974          9.0
4               12 Angry Men             1957          9.0
..                       ...              ...          ...
95                   Vertigo             1958          8.3
96       Singin' in the Rain             1952          8.3
97              Citizen Kane             1941          8.3
98                         M             1931          8.3
99                   The Kid             1921          8.3

[100 rows x 3 columns]


# 3. IMDb top 100 Indian Movies

In [15]:
def Top100IndianMovies(web_url):
 
    list_of_titles = []
    list_of_years = []
    user_ratings = []
 
    source = requests.get(web_url).text
    soup = BeautifulSoup(source, 'lxml')
 
    print(soup.title.text)
 
    tbody = soup.tbody
    table_rows = tbody.find_all('tr')
 
    for td in table_rows:
        titles_column = td.find_all('td', class_="titleColumn")
        imdb_ratings = td.find_all('td', class_="ratingColumn imdbRating")
  
        for information in titles_column:
            title = information.a.text
            list_of_titles.append(title)
            year = information.span.text
            list_of_years.append(year)
        
        for rating in imdb_ratings:
            user_rating = rating.text.strip()
            user_ratings.append(user_rating)
        
    #saving the data in Dataframe
    movies = pd.DataFrame({'MovieName':list_of_titles,
                       'Year of Release':list_of_years,
                       'IMDB Rating':user_ratings,
                       })

    # Cleaning 'year' column
    movies['Year of Release'] = movies['Year of Release'].str.extract('(\d+)').astype(int)
    print(movies[0:100])
    movies.to_csv("Top100IndianMovies.csv")        

In [17]:
Top100IndianMovies("https://www.imdb.com/india/top-rated-indian-movies/")

Top Rated Indian Movies - IMDb
                     MovieName  Year of Release IMDB Rating
0              Pather Panchali             1955         8.5
1                     Gol Maal             1979         8.5
2                      Nayakan             1987         8.5
3                   Anbe Sivam             2003         8.5
4                   Drishyam 2             2021         8.5
..                         ...              ...         ...
95  The Legend of Bhagat Singh             2002         8.0
96                  Bommarillu             2006         8.0
97                     Maqbool             2003         8.0
98                      Bombay             1995         8.0
99                      Omkara             2006         8.0

[100 rows x 3 columns]


# 4. Scraping from ‘www.bookpage.com’

In [18]:
#book name, author name, genre and book review of any 5 books

In [59]:
def bookPage(url):
    
    Book_Name=[]
    Author_Name=[]
    Genre=[]
    b_Review=[]
    
    # Getting English translated titles from the movies
    headers = {'Accept-Language': 'en-US, en;q=0.5'}
    
    if requests.get(url):
        page=requests.get(url,headers=headers)
        print(page)
        soup=BeautifulSoup(page.text,'html.parser')
        
        book_div=soup.find_all('div',class_='flex-article-content')
        
        for container in book_div:
            #scraping book name
            bname=container.h4.a.text
            Book_Name.append(bname)
            
            # Scraping the author name
            aName = container.find('p', class_='sans bold').text
            Author_Name.append(aName)
            #Book_Name.strip()
            
            #scraping genre
            genre=container.find('p',class_='genre-links hidden-phone')
            Genre.append(genre)
            #Genre.strip()
            
            #scraping review
            review=container.find('p',class_='excerpt')
            b_Review.append(review)
            #b_Review.strip()
            
        books=pd.DataFrame({'Book Name':Book_Name ,'Author Name':Author_Name,
                           'Genre' : Genre,'Reviews': b_Review})    
        
        print(books)
    else:
        print("Web Access Denied")

In [60]:
bookPage("https://bookpage.com/reviews?book_genre=fiction&page=1")

<Response [200]>
                                  Book Name  \
0        Brother, Sister, Mother, Explorer    
1             The Sweet Taste of Muscadines   
2                     ★ We Begin at the End   
3                      Creatures of Passage   
4                             Body of Stars   
5                                  My Heart   
6                       The Arsonists' City   
7  The Phone Booth at the Edge of the World   
8                         ★ The Seed Keeper   
9                   ★ How Beautiful We Were   

                                    Author Name  \
0                            \nJamie Figueroa\n   
1                              \nPamela Terry\n   
2                            \nChris Whitaker\n   
3                             \nMorowa Yejidé\n   
4                      \nLaura Maylene Walter\n   
5  \nSemezdin Mehmedinovic, Celia Hawkesworth\n   
6                                \nHala Alyan\n   
7             \nLaura Imai Messina, Lucy Rand\n   
8     

# 5. (i) Scraping Top 10 ODI Teams

In [41]:
def Top10ODIMenTeam(url):
    
    Team_Name = []
    Matches = []
    Points = []
    Ratings=[]
    
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')
 
    print(soup.title.text)
 
    tbody = soup.tbody
    table_rows = tbody.find_all('tr')
 
    for td in table_rows:
        Team_col = td.find_all('span', class_="u-hide-phablet")
        Match_col = td.find_all('td', class_="table-body__cell u-center-text")
        Point_col= td.find_all('td', class_="table-body__cell u-center-text")
        Ratings_col= td.find_all('td', class_="table-body__cell u-text-right rating")
 
        for i in Team_col:
            Tname = i.text
            Team_Name.append(Tname)
            del Team_Name[19:]
        
        for i in Match_col:
            
            Matches.append(Match_col[0].text)
            Points.append(Match_col[1].text)
            Match_col.pop(0)
            del Points[19:38]
        
                    
        for i in Ratings_col:
            rating=i.text.strip()
            Ratings.append(rating)
            
            
       
    #saving the data in Dataframe
    teams = pd.DataFrame({'Team_Name':Team_Name,
                       'YMatches':Matches,
                       'Points':Points,'Ratings':Ratings
                       })

    print(teams[0:10])
    

In [42]:
Top10ODIMenTeam("https://www.icc-cricket.com/rankings/mens/team-rankings/odi")

ICC Ranking for ODI teams International Cricket Council
      Team_Name YMatches Points Ratings
0       England       52  6,102     117
1         India       33  3,857     117
2   New Zealand       39  4,344     111
3     Australia       31  3,345     108
4  South Africa       35  3,490     100
5      Pakistan       38  3,432      90
6    Bangladesh       42  3,372      80
7     Sri Lanka       49  3,802      78
8   West Indies       31  1,844      59
9   Afghanistan       29  1,316      45


# 5. (ii) scraping top 10 Batsman in ODI 

In [39]:
def Top10BatsmanODI(url):
    Batsman_Name = []
    Team = []
    Ratings=[]
    
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')
 
    print(soup.title.text)
 
    tbody = soup.tbody
    table_rows = tbody.find_all('tr')
 
    for td in table_rows:
        batsman_col = td.find_all('td', class_="table-body__cell name")
        Team_col = td.find_all('span', class_="table-body__logo-text")
        Ratings_col= td.find_all('td', class_="table-body__cell u-text-right rating")
 
        for i in batsman_col:
            name = i.a.text
            Batsman_Name.append(name)
            
        for i in Team_col:
            team=i.text.strip()
            Team.append(team)
            
        for i in Ratings_col:
            rating=i.text.strip()
            Ratings.append(rating)
            
            
     
    #saving the data in Dataframe
    batsman = pd.DataFrame({'Batsman Name':Batsman_Name,
                       'Team':Team,
                       'Ratings':Ratings
                       })

    print(batsman)

In [40]:
Top10BatsmanODI("https://www.icc-cricket.com/rankings/mens/player-rankings/odi")

ICC ODI Match Player Rankings International Cricket Council
          Batsman Name Team Ratings
0         Rohit Sharma  IND     842
1           Babar Azam  PAK     837
2          Ross Taylor   NZ     818
3          Aaron Finch  AUS     791
4  Francois du Plessis   SA     790
5         David Warner  AUS     773
6            Shai Hope   WI     773
7      Kane Williamson   NZ     765
8      Quinton de Kock   SA     755


# 5. (iii) scraping top 10 bowlers in ODI

In [23]:
def Top10BowlersODI(url):
    BowlerName = []
    Team = []
    Ratings=[]
    
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')
 
    print(soup.title.text)
 
    tbody = soup.tbody
    table_rows = tbody.find_all('tr')
    main_div=soup.find_all('div',{"class":"rankings-block__container","data-cricket-role":"bowling"})
        
    for td in main_div:
        bowler_col = td.find_all('td', class_="table-body__cell name")
        Team_col = td.find_all('span', class_="table-body__logo-text")
        Ratings_col= td.find_all('td', class_="table-body__cell u-text-right rating")
 
        for i in bowler_col:
            name = i.a.text
            BowlerName.append(name)
            
        for i in Team_col:
            team=i.text.strip()
            Team.append(team)
            
        for i in Ratings_col:
            rating=i.text.strip()
            Ratings.append(rating)
            
            
    #saving the data in Dataframe
    bowlers = pd.DataFrame({'Bowler Name':BowlerName,
                       'Team':Team,
                       'Ratings':Ratings
                       })

    print(bowlers[0:10])

In [24]:
Top10BowlersODI("https://www.icc-cricket.com/rankings/mens/player-rankings/odi")

ICC ODI Match Player Rankings International Cricket Council
         Bowler Name Team Ratings
0   Mujeeb Ur Rahman  AFG     708
1     Jasprit Bumrah  IND     700
2       Mehedi Hasan  BAN     694
3       Chris Woakes  ENG     675
4      Kagiso Rabada   SA     665
5     Josh Hazlewood  AUS     660
6  Mustafizur Rahman  BAN     658
7      Mohammad Amir  PAK     647
8        Pat Cummins  AUS     646
9   Mujeeb Ur Rahman  AFG     708


# 6. (i) scraping top 10 ODI teams (women)

In [37]:
def Top10ODIWomenTeam(url):
    
    Team_Name = []
    Matches = []
    Points = []
    Ratings=[]
    
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')
 
    print(soup.title.text)
 
    tbody = soup.tbody
    table_rows = tbody.find_all('tr')
 
    for td in table_rows:
        Team_col = td.find_all('td', class_="table-body__cell rankings-table__team")
        Match_col = td.find_all('td', class_="table-body__cell u-center-text")
        Point_col= td.find_all('td', class_="table-body__cell u-center-text")
        Ratings_col= td.find_all('td', class_="table-body__cell u-text-right rating")
 
        for i in Team_col:
            Tname = i.text.strip()
            Team_Name.append(Tname)
            #del Team_Name[19:]
        
        for i in Match_col:
            
            Matches.append(Match_col[0].text)
            Points.append(Match_col[1].text)
            Match_col.pop(0)
            #del Points[19:38]
        
                    
        for i in Ratings_col:
            rating=i.text.strip()
            Ratings.append(rating)
            
            
        
    #saving the data in Dataframe
    teams = pd.DataFrame({'Team_Name':Team_Name,
                       'YMatches':Matches,
                       'Points':Points,'Ratings':Ratings
                       })

    print(teams[0:10])
    

In [38]:
Top10ODIWomenTeam("https://www.icc-cricket.com/rankings/womens/team-rankings/odi")

ICC Ranking for ODI teams International Cricket Council
          Team_Name YMatches Points Ratings
0  South Africa\nSA       24  2,828     118
1      England\nENG       17  1,993     117
2        India\nIND       20  2,226     111
3   New Zealand\nNZ       18  1,696      94
4   West Indies\nWI       12  1,025      85
5     Pakistan\nPAK       15  1,101      73
6   Bangladesh\nBAN        5    306      61
7     Sri Lanka\nSL       11    519      47
8      Ireland\nIRE        2     25      13


# 6. (ii) scraping top 10 women ODI players

In [35]:
def Top10WomenODI(url):
    Batsman_Name = []
    Team = []
    Ratings=[]
    
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')
 
    print(soup.title.text)
 
    tbody = soup.tbody
    table_rows = tbody.find_all('tr')
 
    for td in table_rows:
        batsman_col = td.find_all('td', class_="table-body__cell name")
        Team_col = td.find_all('span', class_="table-body__logo-text")
        Ratings_col= td.find_all('td', class_="table-body__cell u-text-right rating")
 
        for i in batsman_col:
            name = i.a.text
            Batsman_Name.append(name)
            
        for i in Team_col:
            team=i.text.strip()
            Team.append(team)
            
        for i in Ratings_col:
            rating=i.text.strip()
            Ratings.append(rating)
            
            
       
    #saving the data in Dataframe
    batsman = pd.DataFrame({'Batsman Name':Batsman_Name,
                       'Team':Team,
                       'Ratings':Ratings
                       })

    print(batsman)

In [36]:
Top10WomenODI("https://www.icc-cricket.com/rankings/womens/player-rankings/odi")

ICC ODI Match Player Rankings International Cricket Council
        Batsman Name Team Ratings
0     Tammy Beaumont  ENG     765
1        Meg Lanning  AUS     749
2    Stafanie Taylor   WI     746
3       Alyssa Healy  AUS     741
4  Amy Satterthwaite   NZ     740
5    Smriti Mandhana  IND     719
6    Laura Wolvaardt   SA     699
7        Mithali Raj  IND     693
8       Ellyse Perry  AUS     691


# 6. (iii) scraping top 10 alrouders ODI

In [45]:
def Top10WomenAlrouderODI(url):
    Alrounder_Name = []
    Team = []
    Ratings=[]
    
    source = requests.get(url).text
    soup = BeautifulSoup(source, 'lxml')
 
    print(soup.title.text)
 
    tbody = soup.tbody
    table_rows = tbody.find_all('tr')
    main_div=soup.find_all('div',{"class":"rankings-block__container","data-cricket-role":"all_round"})
        
    for td in main_div:
        Alrounder_col = td.find_all('td', class_="table-body__cell name")
        Team_col = td.find_all('span', class_="table-body__logo-text")
        Ratings_col= td.find_all('td', class_="table-body__cell u-text-right rating")
 
        for i in Alrounder_col:
            name = i.a.text
            Alrounder_Name.append(name)
            
        for i in Team_col:
            team=i.text.strip()
            Team.append(team)
            
        for i in Ratings_col:
            rating=i.text.strip()
            Ratings.append(rating)
            
            
    #saving the data in Dataframe
    Alrounder = pd.DataFrame({'Alrounder Name':Alrounder_Name,
                       'Team':Team,
                       'Ratings':Ratings
                       })

    print(Alrounder[0:10])

In [46]:
Top10WomenAlrouderODI("https://www.icc-cricket.com/rankings/womens/player-rankings/odi")

ICC ODI Match Player Rankings International Cricket Council
     Alrounder Name Team Ratings
0   Stafanie Taylor   WI     410
1    Marizanne Kapp   SA     390
2     Deepti Sharma  IND     357
3    Natalie Sciver  ENG     349
4     Jess Jonassen  AUS     301
5     Sophie Devine   NZ     274
6  Dane van Niekerk   SA     252
7   Katherine Brunt  ENG     236
8  Ashleigh Gardner  AUS     223


# 7.  scraping Product Name, Price, Image URL and Average Rating 

In [242]:
def PhonesBelow20000(url):
    
    import re
    Product_Name=[]
    Price=[]
    Image=[]
    Ratings=[]
    
    # Getting English translated titles from the movies
    headers = {'Accept-Language': 'en-US, en;q=0.5'}
    
    if requests.get(url):
        page=requests.get(url,headers=headers)
        print(page)
        soup=BeautifulSoup(page.text,'html.parser')
        
        product_div=soup.find_all('div',class_='a-section a-spacing-medium')
        
        for container in product_div:
            #scraping product name
            pname=container.find('span',class_="a-size-medium a-color-base a-text-normal")
            Product_Name.append(pname)
            
            # Scraping the price
            price = container.find('span', class_='a-price-whole')
            Price.append(price)
            #Book_Name.strip()
            
            #scraping image
            # Extract and store in top_items according to instructions on the left
            images = container.select('img',class_="s-image")
            for image in images:
                src = image.get('src')
                alt = image.get('alt')
                Image.append({"src": src,"alt":alt})
                #images = container.find('div', class_="s-image")['style':'url']
                #Image.append(images)
            
            ratings=container.find('span',class_='a-icon-alt')
            Ratings.append(ratings)
            #Genre.strip()
            
        print(len(Product_Name),len(Price),len(Image),len(Ratings)) 
        phones=pd.DataFrame({'Product Name':Product_Name ,'Price':Price,
                           'Image URL' : Image,'Ratings': Ratings})    
        
        print(phones)
    else:
        print("Web Access Denied")

In [244]:
PhonesBelow20000("https://www.amazon.in/s?k=mobiles+under+20000&ref=nb_sb_noss_2")

<Response [200]>
24 24 24 24
                                         Product Name     Price  \
0   [Redmi 9A (Nature Green, 2GB Ram, 32GB Storage...   [6,799]   
1         [Redmi 9 (Sky Blue, 4GB RAM, 64GB Storage)]   [8,799]   
2   [Redmi Note 9 Pro Max (Interstellar Black, 6GB...  [14,999]   
3                                                None      None   
4                                                None      None   
5                                                None      None   
6                                                None      None   
7                                                None      None   
8                                                None      None   
9                                                None      None   
10                                               None      None   
11  [Samsung Galaxy M31 (Ocean Blue, 6GB RAM, 128G...  [16,499]   
12  [Redmi 9A (Nature Green, 3GB Ram, 32GB Storage...   [7,499]   
13  [Redmi Note 9 (Pebble Grey, 4

In [238]:
# Most of the time , access is denied for amazon.in. 
page=requests.get("https://www.amazon.in/s?k=mobiles+under+20000&ref=nb_sb_noss_2")
page

<Response [503]>

# 8. Weather forecast of San Francisco

In [246]:
# period, short description, temperature and description

In [57]:
def WeatherForecast(url):
    
    Period=[]
    ShortDesc=[]
    Temprature=[]
    Description=[]
    
    # Getting English translated titles from the movies
    headers = {'Accept-Language': 'en-US, en;q=0.5'}
    
    if requests.get(url):
        page=requests.get(url,headers=headers)
        print(page)
        soup=BeautifulSoup(page.text,'html.parser')
        
        weather_div=soup.find_all('li',class_='forecast-tombstone')
        detailDesc_div=soup.find_all('div',class_='row row-odd row-forecast')
        for container in weather_div:
            #scraping day
            period=container.find('p',class_='period-name')
            Period.append(period)
            del Period[7:]
            
            # Scraping the ShortDescription
            sDesc = container.find('p', class_='short-desc')
            ShortDesc.append(sDesc)
            del ShortDesc[7:]
            
            #scraping Temprature
            temp=container.find('p',class_={'temp temp-high','temp temp-low'})
            Temprature.append(temp)
            del Temprature[7:]
        
        for i in detailDesc_div:
    
            #scraping Description
            desc=i.find('div',class_='col-sm-10 forecast-text')
            Description.append(desc)
            #del Description[8:] 
            
        print(len(Period),len(ShortDesc),len(Temprature),len(Description))
            
        weather=pd.DataFrame({'Day':Period ,'Short Description':ShortDesc,
                           'Temperature' : Temprature,'Description': Description})    
        print(weather)
    else:
        print("Web Access Denied")

In [58]:
WeatherForecast("https://forecast.weather.gov/MapClick.php?lat=37.777120000000025&lon=-122.41963999999996#.YFg36JMzau4")

<Response [200]>
7 7 7 7
                      Day Short Description    Temperature  \
0         [Today, [], []]    [Mostly Sunny]  [High: 65 °F]   
1       [Tonight, [], []]    [Mostly Clear]   [Low: 47 °F]   
2       [Tuesday, [], []]           [Sunny]  [High: 69 °F]   
3    [Tuesday, [], Night]           [Clear]   [Low: 49 °F]   
4     [Wednesday, [], []]           [Sunny]  [High: 70 °F]   
5  [Wednesday, [], Night]    [Mostly Clear]   [Low: 49 °F]   
6      [Thursday, [], []]           [Sunny]  [High: 65 °F]   

                                         Description  
0  [Mostly sunny, with a high near 65. West wind ...  
1  [Sunny, with a high near 69. North wind 7 to 1...  
2  [Sunny, with a high near 70. Light and variabl...  
3                      [Sunny, with a high near 65.]  
4                      [Sunny, with a high near 66.]  
5                      [Sunny, with a high near 69.]  
6                      [Sunny, with a high near 72.]  
